# Analiza Wielowymiarowa - Zajecia 4 - Analiza kanoniczna

In [ ]:
import os
os.getcwd() # oczekiwany .../AWXXXX/materialy/zajecia04
# mozna uzyc os.chdir("path") do zmiany

In [ ]:
import yaml
spec =  yaml.safe_load(open('../../spec.yaml'))

In [ ]:
# STATA
import stata_setup
stata_setup.config(spec["stata_path"], spec["stata_type"])
from pystata import stata

In [ ]:
import pandas as pd
import scipy
import numpy as np
from statsmodels.multivariate.cancorr import CanCorr
from sklearn.cross_decomposition import CCA
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import pyAesCrypt

In [ ]:
def corr_mat(X1, X2) -> np.matrix:
    """Correlation matrix between two different matries.
    Arguments:
    X1 -- array_like first matrix 
    X2 -- array_like second matrix
    Returns:
        Correlation matrix.
    """
    assert X1.shape[0] == X2.shape[0], "X1 and X2 should have the same number of observations."
    
    col_nams = X2.columns if hasattr(X2, "columns") else list(range(X2.shape[1]))
    index_nams = X1.columns if hasattr(X1, "columns") else list(range(X1.shape[1]))

    X1 = np.asmatrix(X1)
    X2 = np.asmatrix(X2)
    numerator = np.matmul(X1.T, X2) / X2.shape[0] - np.outer(np.mean(X1, axis = 0), np.mean(X2, axis = 0))
    denominator = np.outer(np.std(X1, axis = 0), np.std(X2, axis = 0))
    res = pd.DataFrame(numerator / denominator)
    res.columns = col_nams
    res.index = index_nams
    return res

from typing import List

def REDUNT(matX, matY, can_corrs, corr_Y_xscores, corr_X_yscores) -> List[pd.DataFrame]:
    """Redundancy for CCA analysis.
    Arguments:
    matX -- array_like egzo variables
    matY -- array_like engo variables
    can_corrs -- array_like 1D canonical correlations
    corr_Y_xscores -- array_like correlation between endo variables and xscores
    corr_X_yscores -- array_like correlation between egzo variables and yscores
    Returns:
        Redunadancy - Average percent of variance in a set of variables explained by their own canonical variate.
    """
    assert matX.shape[0] == matY.shape[0], "matX and matY should have the same number of observations."
    assert corr_Y_xscores.shape[1] == corr_X_yscores.shape[1], "corr_Y_xscores and corr_X_yscores should have the same number of columns."
    assert len(can_corrs) >= corr_Y_xscores.shape[1], "can_corrs should have number of elements at least as number of columns in corr_Y_xscores."
    assert len(can_corrs) >= corr_X_yscores.shape[1], "can_corrs should have number of elements at least as number of columns in corr_X_xscores."

    matX = np.asmatrix(matX)
    matY = np.asmatrix(matY)
    can_corrs = np.array(can_corrs)
    corr_Y_xscores = np.asmatrix(corr_Y_xscores)
    corr_X_yscores = np.asmatrix(corr_X_yscores)

    eigenmatY = can_corrs
    vector1 = np.power(eigenmatY, 2)
    names1 = ["own variance", "opposite variance"]
    names2 = ["y", "x"]
    matim = list()
    for i in range(corr_Y_xscores.shape[1]):
        a = np.sum(np.power(corr_Y_xscores[:, i], 2)) / matY.shape[1]
        b = a / vector1[i]
        c = np.sum(np.power(corr_X_yscores[:, i], 2)) / matX.shape[1]
        d = c / vector1[i]
        mm = pd.DataFrame(np.matrix([[b, a], [d, c]]))
        mm.index = names2
        mm.columns = names1
        matim.append(mm)
    return(matim)

In [ ]:
if not os.path.isfile("../../dane/zadowolenie.dta"):
    password = spec["password_pyaescrypt"]
    if password is None:
        password = input("password: ")
    pyAesCrypt.decryptFile("../../dane/zadowolenie.dta.aes", "../../dane/zadowolenie.dta", password)

In [ ]:
%stata use ../../dane/zadowolenie.dta, clear

In [ ]:
zadowolenie = pd.read_stata("../../dane/zadowolenie.dta")

Ziarno losowania

In [ ]:
np.random.seed(0)

### Wielkość Efektu, p-value i liczebność próby

In [1]:
# AMERICAN STATISTICAL ASSOCIATION RELEASES STATEMENT ON STATISTICAL SIGNIFICANCE AND P-VALUES
# https://www.amstat.org/asa/files/pdfs/p-valuestatement.pdf

In [ ]:
# Przypisy na https://pl.wikipedia.org/wiki/Wielko%C5%9B%C4%87_efektu
corr = 0.1       # correlation
covs = [[1 , corr], 
        [corr,  1]] 
means = [0, 0]

In [ ]:
m = np.random.multivariate_normal(means, covs, 100).T
scipy.stats.pearsonr(m[0], m[1])

In [ ]:
m = np.random.multivariate_normal(means, covs, 1000).T
scipy.stats.pearsonr(m[0], m[1])

In [ ]:
m = np.random.multivariate_normal(means, covs, 10000).T
scipy.stats.pearsonr(m[0], m[1])

### CCA - Canonical-Correlation Analysis 

    Copyright (C) 2019  Dorota Celinska-Kopczynska <dot at mimuw dot edu dot pl> Maciej Nasinski

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
    


Do zobrazowania idei analizy kanonicznej przeprowadzimy analize analogiczna do Celinska, Olszewski [2013].

Wykorzystamy baze danych "zadowolenie.dta" oparta na bazie z badania Diagnoza Spoleczna.  
W badaniu wykorzystujemy jedynie obserwacje dotyczace glow gospodarstwa domowego

|typ|zmienna | opis|
|---|----------|---------|
||numer_gd | do diagnostyki, nr gospodarstwa|
|x|plec | plec respondenta (1 -- kobieta, 0 -- mezczyzna) |
|x|wiek2011 | wiek w roku 2011  |
|x|edukacja | wyksztalcenie respondenta  |
|x|stanciv |stan cywilny  |
|x|dochod | sredni (estymowany przez repondenta) dochod z ostatnich 3 miesiecy  |
|x|zaufanie | czy respondent czuje sie kochany i darzony zaufaniem (1 -- tak, 0 -- nie)  |
|x|dep_x | wskazniki z testu Becka dotyczace oznak depresji: wyglad, zapal do pracy, sen, meczenie sie, zdrowie  |
|x|zaleznosc | kto ma wplyw na zycie respondenta (1 -- jestem kowalem swojego losu; 0 -- moje zycie zalezy od wladz, od innych ludzi, od losu opatrznosci)  |
|x|kontakty | liczba kontaktow spolecznych miesiecznie  |
|x|aktywnosc | liczba przypadkow aktywnosci spolecznej (wizyty w kinie, teatrze, na koncertach; restauracjach,ckawiarniach, pubach; spotkania towarzyskie)|
|||
|y|rodzina | zadowolenie ze stosunkow z rodzina  |
|y|przyjaciele | zadowolenie ze stosunkow ze znajomymi  |
|y|zdrowie | zadowolenie ze stanu zdrowia  |
|y|sukces | zadowolenie z osiagniec zyciowych|

Zbior zmiennych zaleznych: rodzina przyjaciele zdrowie sukces  
Zbior zmiennych niezaleznych: plec wiek2011, edukacja, stanciv, dochod, zaufanie, dep_x, zaleznosc, kontakty, aktywnosc

Zmienne dotyczace zadowolenia z zycia sa zakodowane w porzadku rosnacym -- im wieksza wartosc zmiennej tym dana osoba jest bardziej zadowolona z danej czesci swojego zycia

### KROK 1 - podstawowe charakterystyki statystyczne i obejrzenie zbioru danych

Tutaj powinnismy jak najwiecej eksperymentowac.   
Pamietajcie tylko że do raportu wybieramy tylko kilka najważniejszych spostrzeżenń, inne można dodać do załącznika ale także z umiarem.  
Należy zbadać rozkłady zmiennych oraz ich inne charakterystyki. Warto posłużyć sie także statystykami opisowymi.  
Dla wszytskich zmiennych biorących udział w modelowaniu oczekujemy w miarę rozkład normalny.  
Dokładniej nie musi to być precyzyjnie rozkład normalny jednak problematyczne dla modelowania sa wartości odstające.  
Zmienne binarne mogą by dołączone do modelowania, należy zaznaczyć że ich nie będziemy normalizować.   

In [ ]:
# obserwacje brakujace
zadowolenie.loc[zadowolenie.isnull().any(axis = 1), :]

In [ ]:
%%stata
codebook rodzina przyjaciele zdrowie sukces

In [ ]:
zadowolenie.apply(lambda x: x if x.dtypes != "category" else x.cat.codes).describe().round(3).T

In [ ]:
%%stata
centile rodzina przyjaciele zdrowie sukces, level(50)
sum rodzina  przyjaciele zdrowie sukces

In [ ]:
%%stata
hist rodzina, name(q)
hist przyjaciele, name(er)
hist zdrowie, name(t)
hist sukces, name(qy)
graph combine q er t qy
graph drop q er t qy

In [ ]:
zadowolenie[["rodzina", "przyjaciele", "zdrowie", "sukces"]].apply(lambda x: x.cat.codes).hist()

In [ ]:
%%stata
centile wiek2011 dochod edukacja kontakty aktywnosc, level(50)
sum wiek2011 dochod edukacja kontakty aktywnosc

In [ ]:
%stata hist dochod, normal

In [ ]:
%stata hist aktywnosc, normal

In [ ]:
%%stata
graph pie, over(stanciv)

In [ ]:
zadowolenie.groupby(["stanciv"])[["stanciv"]].count().plot.pie(y = "stanciv")

In [ ]:
%%stata
tabulate stanciv
tab plec
tab zaufanie

In [ ]:
zadowolenie.plec.value_counts(),\
zadowolenie.edukacja.value_counts(),\
zadowolenie.stanciv.value_counts(),\
zadowolenie.rodzina.value_counts()

### KROK 2 -- obejrzyjmy korelacje!

Czym silniejsze korelacje pomiedzy dwoma wyroznionymi zbiorami zmiennych (y vs x) tym lepszy efekt da analiza kanoniczna.  
Z drugiej strony korelacje na przestrzeni zbioru y oraz x (z osobna), powinny byc umiarkowane.

Zmienne zalezne sa typowo porzadkowe, wiec uzywamy wspolczynnika kendala.  
Korelacje pomiędzy zmiennymi zależnymi  

In [ ]:
%%stata
ktau rodzina przyjaciele zdrowie sukces, stats(taub p)

In [ ]:
# Macierz jest pusta bo metoda nie działa dla zmiennych typu category
zadowolenie[["rodzina", "przyjaciele", "zdrowie", "sukces"]].corr(method = "kendall").shape

In [ ]:
# Transformacja zmiennych
xmat = zadowolenie[["wiek2011", "edukacja", "stanciv", "dochod", "zaleznosc", "kontakty", "aktywnosc", "plec", "zaufanie"]].\
apply(lambda x: x if x.dtypes != "category" else x.cat.codes)
ymat = zadowolenie[["rodzina", "przyjaciele", "zdrowie", "sukces"]].apply(lambda x: x.cat.codes)

In [ ]:
ymat.corr(method = "kendall").round(3)

Zmienne nie wykazuja relatywnie bardzo silnej korelacji.

Korelacje pomiędzy zmiennymi niezaleznymi.  
Nie cechowaly sie rozkladem normalnym, wiec dla bezpieczenstwa wykorzystamy korelacje spearman-a.

In [ ]:
%%stata
spearman wiek2011 edukacja stanciv dochod zaleznosc kontakty aktywnosc plec zaufanie

Najwyzsze korelacje - lata nauki a wiek, dochod a lata nauki - nie sa one dziwne, jednoczesnie nie przekraczaja 0.5.   
Mozna przyjać, że w zbiorze zmiennych niezaleznych nie ma "niebezpiecznie" istotnych korelacji.

In [ ]:
xmat.corr(method = "spearman").round(3)

Korelacje pomiędzy zbiorami y oraz x

In [ ]:
corr_mat(ymat, xmat).round(3)

### KROK 3 -- analiza kanoniczna

Przechodzimy do analizy pomiedzy zbiorem zmiennych zaleznych a zbiorem zmiennych
niezaleznych - narzedzie: analiza kanoniczna.

Narzędzia:
- STATA: canon 
- PYTHON: statsmodels.multivariate.cancorr.CanCorr oraz sklearn.cross_decomposition.CCA

Y - liczba zmiennych zaleznych: 4  
X - liczba zmiennych niezaleznych: 14  

Dla kazdego zbioru danych mogą zostać utworzone maksymalnie cztery zmienne kanoniczne.   
Minimum z wyjsciowej liczby zmiennych w kazdym ze zbiorow zmiennych wyjsciowych - min(4, 14) = 4

pierwszy zbior analizowanych zmiennych; bedziemy interpretowac go jako zbior zmiennych zaleznych  
drugi zbior analizowanych zmiennych; bedziemy interpretowac go jako zbior zmiennych niezaleznych  

STRANDARYZACJA jest bardzo ważnym krokiem procedury:
- W STATA UZYWAM ZESTANDARYZOWANYCH WAG  -- opcja stdcoef   
- W PYTHONIE STANDARYZUJEMY DANE WEJSCIOWE

#### 3.1 Diagnostyka -- ile par zmiennych kanonicznych wybrac?  Testy 

In [ ]:
zadowolenie = zadowolenie.apply(lambda x: x.cat.codes if x.dtypes == "category" else x)

Model bez zmiennych binarnych, można porównać z modelem który je zawiera.  
Wyniki nie wydają się bardzo różnić, zwłaszcza dla 2 pierwszych zmiennych kanonicznych.

In [ ]:
#%%stata
#xi:canon (rodzina przyjaciele zdrowie sukces) (dep_wyglad dep_zapal dep_zdrowie dep_sen dep_meczenie wiek2011 kontakty aktywnosc edukacja dochod) , test(1 2 3 4) stdcoef

Model ze zmiennymi binarnymi

In [ ]:
%%stata
xi:canon (rodzina przyjaciele zdrowie sukces) (dep_wyglad dep_zapal dep_zdrowie dep_sen dep_meczenie i.plec wiek2011 kontakty aktywnosc edukacja i.zaufanie i.zaleznosc i.stanciv dochod) , first(2) test(1 2 3 4) stdcoef

---------------------------------------------

Przygotowanie danych dla modelowania w pythonie

In [ ]:
# rozbicie zmiennych typu category na zmienne binarne
zadowolenie[["plec", "zaleznosc", "stanciv", "zaufanie"]] = zadowolenie[["plec", "zaleznosc", "stanciv", "zaufanie"]].astype("int")
zadowolenie_cols = pd.get_dummies(zadowolenie, columns = ["plec", "zaleznosc", "stanciv", "zaufanie"], drop_first=True)

In [ ]:
# standaryzacja bez zmiennych binarnych
norm_cols = ["rodzina", "przyjaciele", "sukces", "zdrowie"] + ["wiek2011", "kontakty", "aktywnosc", "edukacja", "dochod", "dep_wyglad", "dep_zapal", "dep_zdrowie", "dep_sen", "dep_meczenie"]
zadowolenie_cols[norm_cols] = (zadowolenie_cols[norm_cols] - zadowolenie_cols[norm_cols].mean())/ zadowolenie_cols[norm_cols].std(ddof=1)

In [ ]:
y_cols = ["rodzina", "przyjaciele", "zdrowie", "sukces"]
x_cols = ["dep_wyglad", "dep_zapal", "dep_zdrowie", "dep_sen", "dep_meczenie", "plec_1", "wiek2011", "kontakty",
            "aktywnosc", "edukacja", "zaufanie_1", "zaleznosc_1", "stanciv_1", "stanciv_2", "stanciv_3", "stanciv_4", "dochod"]
y_mat = zadowolenie_cols[y_cols]
x_mat = zadowolenie_cols[x_cols]

------------------------

#### Walidacja ilościowa modelu. - wykorzystana tylko w pythonie. 
Do tego celu wykorzystamy Cross-validacje oraz przeszukanie siatki.  
Warto zwrócić uwagę że próbę dzielimy na uczącą (train) oraz testową, dzięki temu możemy otrzymać wiarygodną ocene dla wybranego modelu.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_mat, y_mat, test_size=0.25, random_state=0)

cca = CCA(copy=True, scale = False)

param_grid = {
    'n_components': [1, 2, 3, 4]
}

search = GridSearchCV(cca, param_grid, n_jobs=-1, )
search.fit(X_train, y_train)
print("Best parameter (CV score={}):".format(search.best_score_))
print(search.best_params_)

search.score(X_test, y_test)

Brzytwa Ockhama - i tym razem prostszy model okazuje się znacznie lepszy pod względem predykcyjnym.  
Model predykcyjny biorący pod uwagę 2 zmienne kanoniczne ma x3 większe R2 od modelu z 4 zmiennymi.

In [ ]:
# Wszytskie wyniki
# search.cv_results_

#### Estymacja Modelu

In [ ]:
#statsmodels
# https://devdocs.io/statsmodels/generated/statsmodels.multivariate.cancorr.cancorr
res = CanCorr(y_mat, x_mat)
# or sklearn
# https://scikit-learn.org/stable/modules/cross_decomposition.html#cross-decomposition
my_cca = CCA(n_components=2, copy = True, max_iter = 100, scale = False)
my_cca.fit(x_mat, y_mat)
# different vars order

In [ ]:
# Predykcja
# Y = X * coef
# coef = np.dot(my_cca.x_rotations_, my_cca.y_loadings_.T)
# my_cca.coef_
# predicitng y with x and coefs
# y_mat, (np.matmul(x_mat, my_cca.coef_))

Zmienne Kanoniczne

In [ ]:
x_scores, y_scores = my_cca.transform(x_mat, y_mat)

In [ ]:
# reproduce x with scores and loadings
# pierwsza obserwacja
# pd.DataFrame({"x_r": np.matmul(x_scores, my_cca.x_loadings_.T)[:1,:].ravel(), "x": StandardScaler().fit_transform(x_mat)[:1,:].ravel()})

In [ ]:
# get scores
# np.matmul(y_mat, my_cca.y_rotations_), y_scores
# and only approximate for this case
# np.matmul(x_mat, my_cca.x_rotations_), x_scores

In [ ]:
res.cancorr, np.diag(corr_mat(x_scores, y_scores))

Wagi - orginalne = zmienne kanoniczne * wagi

In [ ]:
y_coef = pd.DataFrame(res.y_cancoef * 100)
y_coef.index = y_mat.columns
x_coef = pd.DataFrame(res.x_cancoef * 100)
x_coef.index = x_mat.columns
y_coef.round(4), x_coef.round(4)

In [ ]:
np.round(my_cca.y_rotations_, 4) , np.round(my_cca.x_rotations_, 4)

In [ ]:
res.corr_test().summary()

----------------

Testowanie istotnosci koreacji kanonicznych.
Najpierw testowana jest H0, ze oba wsp. korelacji kanonicznych sa nieistotne
wobec alternatywy, ze przynajmniej pierwsza para zmiennych kanonicznych jest istotna
- st. testowa = 121.64 i p-value < .0001.
Odrzucamy H0 -
czyli pierwsza korelacja kanoniczna jest istotnie rozna od zera.

Nastepnie testowana jest istotnosc drugiego wsp. korelacji kanonicznej - st. testowa = 51.20
i p-value < .0001. Odrzucamy H0 - czyli druga korelacja kanoniczna jest rowniez
istotnie rozna od zera.

Nastepnie testowana jest istotnosc trzeciego wsp. korelacji kanonicznej - st. testowa = 22.21
i p-value < .0001. Odrzucamy H0 - czyli trzecia korelacja kanoniczna jest rowniez
istotnie rozna od zera.

Nastepnie testowana
jest istotnosc czwartego wsp. korelacji kanonicznej - st. testowa = 19.23
i p-value < .0001. Odrzucamy H0 - czyli czwarta korelacja kanoniczna jest rowniez
istotnie rozna od zera.

Wniosek: wszystkie wsp. korelacji kanonicznej sa istotnie rozne od zera.

Omowiona procedura moze byc wazna wskazowka w przypadku analiz na wiekszych
zbiorach danych i wyboru najwazniejszych (istotnych) zmiennych kanonicznych
do dalszej analizy.  
Test wskazuje na ktorym etapie mozna przyjac, ze pozostale
(najmniejsze) wsp. korelacji kanonicznej sa nieistotnie rozne od zera.

--------------------------

#### 3.2 Wspolczynniki korelacji kanonicznej
I wsp korelacj kanonicznej = 0.6652. Jest to najwiekszy mozliwy do osiagniecia
wsp. korelacji miedzy liniowymi kombinacjami zmiennych z obu zbiorow. Wielkosc ta jest
wieksza niz kazdy wsp. korelacji miedzy zmienna ze zbioru X, a zmienna ze zbioru
Y.

#### 3.3. Wagi kanonicze - pokazuja wklad poszczegolnych zmiennych do zmiennych kanonicznych.
Poniewaz wyjsciowe zmienne sa mierzone na roznych skalach oraz maja rozna wariancje
nalezy interpretowac "standaryzowane wagi kanoniczne"

Ze wzgledu na operowanie wystandaryzowanymi wartosciami zmiennych wejsciowych odpowiadaja one wspolczynnikom
beta w regresji wielorakiej.
Czym wieksza wartosc bezwzgledna wagi danej zmiennej wejsciowej tym wiekszy jej wklad do danej zmiennej kanonicznej

#### 3.4. Standaryzowane wagi kanoniczne - wagi kanoniczne przemnozone przez odchylenia
standardowe wyjsciowych zmiennych. Wspolczynniki te sa wykorzystywane do interpretacji
uzyskanych liniowych kombinacji (zmiennych kanonicznych).

In [ ]:
scores_corr_X_yscores = corr_mat(x_mat, y_scores)
scores_corr_X_xscores = corr_mat(x_mat, x_scores)
scores_corr_Y_xscores = corr_mat(y_mat, x_scores)
scores_corr_Y_yscores = corr_mat(y_mat, y_scores)

In [ ]:
#%%stata
#estat correlations

#### 3.5. INTERPRETACJA

X_WEIGHTS = X_scores vs X - X_scores =  X * X_WEIGHTS  
Y_WEIGHTS = Y_scores vs Y - Y_scores = Y * Y_WEIGHTS

Wybor jest subiektywny -- wybieramy kilka zmiennych o najwyzszych co do wartosci bezwglednej wartosciach wspolczynnikow

Pierwsza zmienna kanoniczna -- postrzeganie zdrowia (bo f63_4 wysokie)
zdrowie -- osoby lepiej oceniajace swoje zdrowie sa bardziej zadowolone ze swojego stanu zdrowia (0.49)
zmeczenie -- osoby mniej odczuwajace zmeczenie sa bardziej zadowolone ze swojego stanu zdrowia (0.24)
zapal do pracy -- osoby majace nie mniej zapalu do pracy niz dawniej sa bardziej zadowolone ze swojego stanu zdrowia (0.17)
zaufanie -- poczucie ze mozna komus zaufac wplywa pozytywnie na poziom zadowolenia (0.13)
wyglad -- lepsza ocena swojego wygladu wiaze sie z wiekszym zadowoleniem  (0.12)

Druga zmienna kanoniczna -- zadowolenie z zycia rodzinnego (0.60), swoich sukcesow (0.60) i niezadowolenie ze stanu zdrowia (-0.61)
bycie kochanym -- osoby czujace sie bardziej kochane sa bardziej zadowolone z zycia rodzinnego ,etc(0.60)
wiek --  osoby starsze sa bardziej zadowolone z zycia rodzinnego ale mniej ze stanu zdrowia (0.58)
zdrowie -- wieksze martwienie sie o zdrowie to mniejsza satysfakcja z niego (-0.34)
wyglad -- lepsze zdanie o swoim wygladzie wiaze sie z wyzszymi wartosciami 2 zmiennej (0.26)
kontakty -- wieksza liczba kontaktow spolecznych wiaze sie z wyzszymi wartosciami 2 zmiennej (0.22)

Tutaj moglismy miec artefakt wplywu wieku, jesli spojrzymy na wplyw stanu cywilnego!

#### 3.6. LOADINGS

X_LOADINGS = X vs X_scores  - X = X_scores * X_LOADINGS  
Y_LOADINGS = Y vs Y_scores  - Y = Y_scores * Y_LOADINGS

Interpretowanie standaryzowanych wag kanonicznych moze byc utrudnione, jezeli
w ktoryms ze zbiorow wyjsciowych zmienych sa zmienne bardzo silnie ze soba skorelowane
(w kontekscie naszego zbioru danych tak nie jest). Np. jezeli w zbiorze X-ow sa
dwie zmienne silnie dodatnio ze soba skorelowane oraz kazda z tych zmiennych jest silnie
dodatnio skorelowana ze zmienna kanoniczna, to moze zdarzyc sie, iz korelacje miedzy
tymi X-ami a zmienna kanoniczna sa dodatnie, natomiast jedna ze standaryzowanych wag
kanonicznych jest dodatnia a druga ujemna. Wowczas zaleca sie intepretowac korelacje
miedzy zmiennymi wyjsciowymi a zmiennymi kanonicznymi.

In [ ]:
%%stata 
/* 3.6 Ladunki kanoniczne*/
estat loadings

In [ ]:
x_loadings = pd.DataFrame(my_cca.x_loadings_)
x_loadings.index = x_mat.columns
y_loadings = pd.DataFrame(my_cca.y_loadings_)
y_loadings.index = y_mat.columns
y_loadings.round(4), x_loadings.round(4)

### 3.7 Redundancja i wariancja wyodrebniona

**Redundancja bardzo podobna do dobrze nam znanego R2 z regresji liniowej**

Interpretacja dla zmiennych niezaleznych.
Wariancja wyodrebniona: dla kazdej zmiennej kanonicznej obliczamy srednia z kwadratow
wsp. korelacji miedzy ta zmienna kanoniczna a zmiennymi z odpowiadajacego jej zbioru
zmiennych.

Redundacja: iloczyn kwadratu korelacji kanonicznej i wariancji wyodrebnionej:

Redundacja mowi nam ile przecietnie wariancji w jednym zbiorze jest wyjasnione przez dana
zmienna kanoniczna  przy danym innym zbiorze zmiennych. Wartosc redundacji calkowitej
moze byc wazna analityczna informacja o naszym modelu. Mowi o procencie calkowitej
wariancji jednego zbioru wyjasniona w ramach modelu. Moze byc pomocna w ocenie, czy
poprzez dolaczenie nowych zmiennych, uzyskamy wiekszy zakres wyjasnienia niz poprzednio.


#### Instalacja pakietu STATA canred

In [ ]:
%stata search canred

In [ ]:
# FROM STATA GUI 
# search canred
# https://stats.idre.ucla.edu/stata/ado/analysis/

In [ ]:
# %stata ssc install canred
# could fail

In [ ]:
%stata sysdir

In [ ]:
# %stata mkdir "/Users/maciejnasinski/Library/Application Support/Stata/ado/plus", public

In [ ]:
# %%stata
# copy /Users/maciejnasinski/Documents/AW/materialy/zajecia04/canred/canred.ado `"`c(sysdir_plus)'canred.ado"', public replace
# discard // you have to discard to see installed adofiles
# which canred

In [ ]:
%%stata
canred 1
/* 1 zmienna kanoniczna wyjasnia przecietnie 38.7% zmiennosci w zbiorze Y w oparciu o X
   Redundancja wynosi 17%*/

canred 2

/* 2 zmienna kanoniczna wyjasnia przecietnie 28% zmiennosci w zbiorze Y w oparciu o X
   Redundancja wynosi 5% */

In [ ]:
REDUNT(x_mat, y_mat, res.cancorr, scores_corr_Y_xscores, scores_corr_X_yscores)

In [ ]:
(red0 := np.sum(np.power(scores_corr_Y_xscores.iloc[:,0], 2))/4,  red0/(res.cancorr[0])**2),\
(red1 := np.sum(np.power(scores_corr_Y_xscores.iloc[:,1], 2))/4, red1/(res.cancorr[1])**2)

### KROK 4 - diagnostyka modelu

In [ ]:
%%stata
/*obserwacje odstajace zapisujemy poszczegolne zmienne kanoniczne */
  
predict u1, u corr(1)
predict u2, u corr(2)
predict v1, v corr(1)
predict v2, v corr(2)

/* Sprawdzamy, czy rzeczywiscie korelacje miedzy zmiennymi kanonicznymi w obrebie jednego
zbioru zmiennych wyjsciowych wynosza 0.*/
correlate u1 u2 v1 v2

In [ ]:
np.round(pd.concat([pd.DataFrame(x_scores), pd.DataFrame(y_scores)], axis = 1).corr(), 3)

szukamy outlierow rysujemy wykresy par zmiennych kanonicznych jako etykiet uzywamy numeru gospodarstwa

In [ ]:
%%stata
scatter u1 v1, mlabel(numer_gd) 

In [ ]:
%%stata
scatter u2 v2, mlabel(numer_gd) 

In [ ]:
#%%stata
#scatter u3 v3, mlabel(numer_gd)

In [ ]:
#%%stata
#scatter u4 v4, mlabel(numer_gd)

### Les Misérables

In [ ]:
%%stata
/* zobaczmy co z nim nie tak */
list  wiek2011 edukacja dochod zaleznosc kontakty aktywnosc rodzina przyjaciele zdrowie sukces plec zaufanie if numer_gd==9095
/* nie jest specjalnie dziwna obserwacja
   w zasadzie mamy tutaj chmurki w kazdym wykresie, wiec nie wyrozniaja sie specjalnie oberwacje odstajace*/

In [ ]:
pd.DataFrame({"y1": y_scores[:,0], "x1": x_scores[:,0]}).plot(x = 'x1', y = 'y1', kind = "scatter", use_index = True)

In [ ]:
y = y_scores[:, 0]
x = x_scores[:, 0]
l = list(range(y_scores.shape[0]))

fig, ax = plt.subplots()
ax.scatter(x, y)

for i, txt in enumerate(l):
    ax.text(x[i], y[i], txt)

In [ ]:
zadowolenie.iloc[883,:]